In [ ]:
from src.attribution.flextime import FLEXtimeMask, Filterbank
from physioex.physioex.data import PhysioExDataModule

In [ ]:
datamodule = PhysioExDataModule(
    datasets=["sleepedf"],     # list of datasets to be used
    batch_size=64,             # batch size for the DataLoader
    preprocessing="raw",       # preprocessing method
    selected_channels=["EEG"],  # channels to be selected
    sequence_length=7,        # length of the sequence
    data_folder="./data",      # path to the data folder
)